### Use resources of GPU to execute these work

#### I used Ollama, llama 3.1 (8B parameters), running localhost, port 11434

In [1]:
# Requeriments

%pip install pandas --quiet
%pip install langchain --quiet
%pip install langchain-community --quiet
%pip install colab-xterm --quiet
%pip install langchain-ollama --quiet

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# Imports

import sqlite3
import pandas as pd

from langchain_ollama import OllamaLLM
from typing_extensions import TypedDict
from langchain.prompts import PromptTemplate
from langchain_community.utilities import SQLDatabase
from langchain_community.tools.sql_database.tool import QuerySQLDatabaseTool

In [4]:
# Create and/or connect in database

conn = sqlite3.connect("./estatisticas_pix.db")

## APRESENTAÇÃO E EXPLICAÇÃO DOS DADOS

### Base de dados extraída do site do branco central:
https://olinda.bcb.gov.br/olinda/servico/Pix_DadosAbertos/versao/v1/aplicacao#!/recursos

---

### Descrição:
Quantidade e volume financeiro de transações Pix liquidadas mensalmente. Não inclui Pix liquidados nos livros do participante, isto é, transações não enviadas para liquidação no SPI. Apenas do mês de novembro de 2024.

---

In [6]:
# Read csv with pandas

df = pd.read_csv("./estatisticas_transacoes_pix.csv", sep=",")

In [7]:
df.head(10)

,AnoMes,PAG_PFPJ,REC_PFPJ,PAG_REGIAO,REC_REGIAO,PAG_IDADE,REC_IDADE,FORMAINICIACAO,NATUREZA,FINALIDADE,VALOR,QUANTIDADE
0,202411,PF,PJ,NORDESTE,NORTE,entre 30 e 39 anos,Nao se aplica,MANU,P2G,Pix,"31488,37",5
1,202411,PF,PF,NORDESTE,SUDESTE,entre 50 e 59 anos,entre 40 e 49 anos,MANU,P2P,Pix,"17702192,83",93119
2,202411,PF,PJ,SUDESTE,SUL,mais de 60 anos,Nao se aplica,DICT,P2G,Pix,"21802,41",39
3,202411,PJ,PF,SUL,SUL,Nao se aplica,entre 50 e 59 anos,DICT,B2P,Pix,"4398553782,39",3242867
4,202411,PF,PJ,NORTE,SUL,entre 40 e 49 anos,Nao se aplica,QRDN,P2G,Pix,"366494,02",796
5,202411,PF,PJ,SUL,CENTRO-OESTE,entre 50 e 59 anos,Nao se aplica,QRES,P2G,Pix,"7374,1",92
6,202411,PF,PF,SUL,NORTE,até 19 anos,entre 30 e 39 anos,QRES,P2P,Pix,"126822,58",5720
7,202411,PF,PF,SUDESTE,Nao informado,entre 20 e 29 anos,até 19 anos,QRES,P2P,Pix,"3790,95",57
8,202411,PF,PF,CENTRO-OESTE,SUDESTE,mais de 60 anos,mais de 60 anos,QRDN,P2P,Pix,"69036,47",1270
9,202411,PF,PJ,NORDESTE,NORDESTE,entre 20 e 29 anos,Nao se aplica,MANU,P2G,Pix,"182825,26",457


In [8]:
# Create table estatisticas_pix and send data from pandas dataframe into table

df.to_sql("estatisticas_pix", conn, if_exists="replace")

11415

In [9]:
# bank access test with query pandas

df_db = pd.read_sql('SELECT * FROM estatisticas_pix', conn)
df_db.head(10)

,index,AnoMes,PAG_PFPJ,REC_PFPJ,PAG_REGIAO,REC_REGIAO,PAG_IDADE,REC_IDADE,FORMAINICIACAO,NATUREZA,FINALIDADE,VALOR,QUANTIDADE
0,0,202411,PF,PJ,NORDESTE,NORTE,entre 30 e 39 anos,Nao se aplica,MANU,P2G,Pix,"31488,37",5
1,1,202411,PF,PF,NORDESTE,SUDESTE,entre 50 e 59 anos,entre 40 e 49 anos,MANU,P2P,Pix,"17702192,83",93119
2,2,202411,PF,PJ,SUDESTE,SUL,mais de 60 anos,Nao se aplica,DICT,P2G,Pix,"21802,41",39
3,3,202411,PJ,PF,SUL,SUL,Nao se aplica,entre 50 e 59 anos,DICT,B2P,Pix,"4398553782,39",3242867
4,4,202411,PF,PJ,NORTE,SUL,entre 40 e 49 anos,Nao se aplica,QRDN,P2G,Pix,"366494,02",796
5,5,202411,PF,PJ,SUL,CENTRO-OESTE,entre 50 e 59 anos,Nao se aplica,QRES,P2G,Pix,"7374,1",92
6,6,202411,PF,PF,SUL,NORTE,até 19 anos,entre 30 e 39 anos,QRES,P2P,Pix,"126822,58",5720
7,7,202411,PF,PF,SUDESTE,Nao informado,entre 20 e 29 anos,até 19 anos,QRES,P2P,Pix,"3790,95",57
8,8,202411,PF,PF,CENTRO-OESTE,SUDESTE,mais de 60 anos,mais de 60 anos,QRDN,P2P,Pix,"69036,47",1270
9,9,202411,PF,PJ,NORDESTE,NORDESTE,entre 20 e 29 anos,Nao se aplica,MANU,P2G,Pix,"182825,26",457


In [10]:
# Creating varaible db and bank access test

db = SQLDatabase.from_uri("sqlite:///estatisticas_pix.db")

print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM estatisticas_pix LIMIT 10;")

sqlite
['estatisticas_pix']


"[(0, 202411, 'PF', 'PJ', 'NORDESTE', 'NORTE', 'entre 30 e 39 anos', 'Nao se aplica', 'MANU', 'P2G', 'Pix', '31488,37', 5), (1, 202411, 'PF', 'PF', 'NORDESTE', 'SUDESTE', 'entre 50 e 59 anos', 'entre 40 e 49 anos', 'MANU', 'P2P', 'Pix', '17702192,83', 93119), (2, 202411, 'PF', 'PJ', 'SUDESTE', 'SUL', 'mais de 60 anos', 'Nao se aplica', 'DICT', 'P2G', 'Pix', '21802,41', 39), (3, 202411, 'PJ', 'PF', 'SUL', 'SUL', 'Nao se aplica', 'entre 50 e 59 anos', 'DICT', 'B2P', 'Pix', '4398553782,39', 3242867), (4, 202411, 'PF', 'PJ', 'NORTE', 'SUL', 'entre 40 e 49 anos', 'Nao se aplica', 'QRDN', 'P2G', 'Pix', '366494,02', 796), (5, 202411, 'PF', 'PJ', 'SUL', 'CENTRO-OESTE', 'entre 50 e 59 anos', 'Nao se aplica', 'QRES', 'P2G', 'Pix', '7374,1', 92), (6, 202411, 'PF', 'PF', 'SUL', 'NORTE', 'até 19 anos', 'entre 30 e 39 anos', 'QRES', 'P2P', 'Pix', '126822,58', 5720), (7, 202411, 'PF', 'PF', 'SUDESTE', 'Nao informado', 'entre 20 e 29 anos', 'até 19 anos', 'QRES', 'P2P', 'Pix', '3790,95', 57), (8, 2024

In [11]:
# Check installation ollama 3.2

!ollama list

NAME               ID              SIZE      MODIFIED       
llama3.1:latest    46e0c10c039e    4.9 GB    44 minutes ago    


In [12]:
# Initialize the OllamaLLM

model_Ollama = OllamaLLM(
    model='llama3.1',  # Specify the model version
    base_url='http://localhost:11434',  # URL where Ollama is running locally
    temperature=0.7  # Control the randomness of the output (0.0 to 1.0)
)

In [13]:
# Creating template to prompt

query_prompt_template = PromptTemplate(
    input_variables=["dialect", "top_k", "table_info", "input", "base_description", "column_description"],
    template=(
        "Você é um assistente especializado em traduzir perguntas feitas em linguagem natural em query SQL.\n\n"
        "Informações do banco de dados:\n"
        "- Descrição da base de dados: {base_description}\n"
        "- Idioma da base de dados: Português\n"
        "- Dialeto SQL: {dialect}\n\n"
        "Estrutura e descrição das tabelas no banco:\n{table_info}\n\n"
        "Descrição completa das colunas:\n{column_description}\n\n"
        "Pergunta do usuário (em português): {input}\n\n"
        "A coluna valor está em reais\n\n"
        "Gere apenas a consulta SQL correspondente\n\n"
        "Abaixo estão alguns exemplos de perguntas e as respectivas queries SQL:\n"
        "- pergunta: Qual foi o volume financeiro total das transações Pix na região Sudeste?\n"
        "- query: SELECT REC_REGIAO AS Regiao_Recebedor, SUM(VALOR) AS Volume_Financeiro_Total FROM transacoes_pix WHERE REC_REGIAO = 'Sudeste' GROUP BY REC_REGIAO;\n"
        "- pergunta: Quantas transações Pix foram realizadas por pessoas físicas?\n"
        "- query: SELECT PAG_PFPJ AS Tipo_Pessoa_Pagador, SUM(QUANTIDADE) AS Quantidade_Transacoes FROM transacoes_pix WHERE PAG_PFPJ = 'Física' GROUP BY PAG_PFPJ;\n"
        "- pergunta: Qual foi o valor médio das transações realizadas por pagadores com idade entre 30 e 40 anos?\n"
        "- query: SELECT PAG_IDADE AS Idade_Pagador, AVG(VALOR) AS Valor_Medio_Transacoes FROM transacoes_pix WHERE PAG_IDADE BETWEEN 30 AND 40 GROUP BY PAG_IDADE;\n"
        "- pergunta: Qual foi a finalidade mais comum das transações?\n"
        "- query: SELECT FINALIDADE AS Finalidade_Transacao, COUNT(*) AS Ocorrencias FROM transacoes_pix GROUP BY FINALIDADE ORDER BY Ocorrencias DESC LIMIT 1;\n\n"
    ),
)

In [14]:
query_prompt_template.pretty_print()

Você é um assistente especializado em traduzir perguntas feitas em linguagem natural em query SQL.

Informações do banco de dados:
- Descrição da base de dados: {base_description}
- Idioma da base de dados: Português
- Dialeto SQL: {dialect}

Estrutura e descrição das tabelas no banco:
{table_info}

Descrição completa das colunas:
{column_description}

Pergunta do usuário (em português): {input}

A coluna valor está em reais

Gere apenas a consulta SQL correspondente

Abaixo estão alguns exemplos de perguntas e as respectivas queries SQL:
- pergunta: Qual foi o volume financeiro total das transações Pix na região Sudeste?
- query: SELECT REC_REGIAO AS Regiao_Recebedor, SUM(VALOR) AS Volume_Financeiro_Total FROM transacoes_pix WHERE REC_REGIAO = 'Sudeste' GROUP BY REC_REGIAO;
- pergunta: Quantas transações Pix foram realizadas por pessoas físicas?
- query: SELECT PAG_PFPJ AS Tipo_Pessoa_Pagador, SUM(QUANTIDADE) AS Quantidade_Transacoes FROM transacoes_pix WHERE PAG_PFPJ = 'Física' GROUP

In [15]:
# Creating base class to generate the structure and types of objects

class State(TypedDict):
    question: str
    query: str
    result: str
    answer: str

In [16]:
# Function to generate SQL query to fetch information

def write_query(state: State):

    base_description = "Base de dados contendo transações Pix, incluindo volume financeiro, quantidade e detalhes regionais."
    column_description = (
        "AnoMes: Data-base no formato ano/mês.\n"
        "PAG_PFPJ: Tipo de pessoa do pagador (Física ou Jurídica).\n"
        "REC_PFPJ: Tipo de pessoa do recebedor (Física ou Jurídica).\n"
        "PAG_REGIAO: Região do pagador.\n"
        "REC_REGIAO: Região do recebedor.\n"
        "PAG_IDADE: Idade do pagador.\n"
        "REC_IDADE: Idade do recebedor.\n"
        "FORMAINICIACAO: Forma de iniciação da transação.\n"
        "NATUREZA: Natureza da transação.\n"
        "FINALIDADE: Finalidade da transação.\n"
        "VALOR: Valor das transações Pix.\n"
        "QUANTIDADE: Quantidade total de transações Pix.\n"
    )

    prompt = query_prompt_template.format(
        dialect=db.dialect,
        top_k=10,
        table_info=db.get_table_info(),
        input=state["question"],
        base_description=base_description,
        column_description=column_description,
    )

    query = model_Ollama.invoke(prompt)
    print(f"Consulta SQL: {query}")
    state["query"] = query
    
    return state

In [17]:
# Function to Execute SQL query

def execute_query(state: State): 

    execute_query_tool = QuerySQLDatabaseTool(db=db)
    result = execute_query_tool.invoke(state["query"])
    print(f"Resuldado SQL: {result}")
    state["result"] = result
    
    return state

In [18]:
# Function to answer question using retrieved information

def generate_answer(state: State):
    
    prompt = (
        "Dada a pergunta do usuário, a consulta SQL correspondente, "
        "e o resultado da consulta SQL, responda à pergunta do usuário de forma clara e objetiva.\n\n"
        f'Pergunta: {state["question"]}\n'
        f'Consulta SQL: {state["query"]}\n'
        f'Resultado SQL: {state["result"]}'
    )
    answer = model_Ollama.invoke(prompt)
    print(f"answer: {answer}")
    state["answer"] = answer

    return state

In [19]:
# Send ask

#ask = "Qual o valor total de transações pix foi recebido pelo nordeste?"
#ask = "Qual valor total foi enviado da região sul para a região norte?"
ask = "Qual a quantidade de transações pix foi realizado entre a região norte e sul?"
#ask = "Quantas transações pix foram feitas da região nordeste por pessoas entre 30 e 59 anos para região norte que ultrapassaram 100 mil reais?"

state = State(
  question=ask,
  query="",
  result="",
  answer=""
)

print()
print("Gerando a query a partir do texto")
write_query(state)

print()
print("Executando a query")
execute_query(state)

print()
print("Gerando resposta para a pergunta em linguagem natural")
generate_answer(state)



Gerando a query a partir do texto
Consulta SQL: SELECT SUM(QUANTIDADE) FROM estatisticas_pix WHERE PAG_REGIAO = 'NORTE' AND REC_REGIAO = 'SUL';

Executando a query
Resuldado SQL: [(18170096,)]

Gerando resposta para a pergunta em linguagem natural
answer: A pergunta do usuário é sobre a quantidade de transações Pix realizadas entre a região Norte e Sul.

A consulta SQL corresponde à essa pergunta, pois está buscando somar a quantidade de transações Pix onde o remetente está na região Norte (PAG_REGIAO = 'NORTE') e o receptor está na região Sul (REC_REGIAO = 'SUL').

E o resultado da consulta SQL é 18170096, o que significa que houve essa quantidade de transações Pix entre a região Norte e Sul.

Portanto, a resposta à pergunta do usuário é: **Foram realizadas 18.170.096 transações Pix entre a região Norte e Sul.**


{'question': 'Qual a quantidade de transações pix foi realizado entre a região norte e sul?',
 'query': "SELECT SUM(QUANTIDADE) FROM estatisticas_pix WHERE PAG_REGIAO = 'NORTE' AND REC_REGIAO = 'SUL';",
 'result': '[(18170096,)]',
 'answer': "A pergunta do usuário é sobre a quantidade de transações Pix realizadas entre a região Norte e Sul.\n\nA consulta SQL corresponde à essa pergunta, pois está buscando somar a quantidade de transações Pix onde o remetente está na região Norte (PAG_REGIAO = 'NORTE') e o receptor está na região Sul (REC_REGIAO = 'SUL').\n\nE o resultado da consulta SQL é 18170096, o que significa que houve essa quantidade de transações Pix entre a região Norte e Sul.\n\nPortanto, a resposta à pergunta do usuário é: **Foram realizadas 18.170.096 transações Pix entre a região Norte e Sul.**"}